In [1]:
import pandas as pd
import os
from subprocess import call
import numpy as np

In [2]:
EXP_DATE='230821'

# List the files in the folder
file_names = [i for i in os.listdir(EXP_DATE) if "~" not in i]
# load the folders:
enzyme_conc=[(EXP_DATE+'/'+i) for i in file_names if 'genex_mt' in i] 
cofactor_conc=[(EXP_DATE+'/'+i) for i in file_names if 'buffers_mt' in i] 
data_file_names = os.listdir(EXP_DATE+'/data')
metab_conc=[(EXP_DATE+'/data/'+i) for i in data_file_names if 'compiled' in i]


In [3]:
def get_most_updated(file_list):
    assert file_list
    if len(file_list) > 1:
        # split the file name by the period in the extension
        versions = [i.split('.')[0].split('-')[-1] for i in file_list]
        versions = [i for i in versions if '/' not in i]
        versions.sort()
        latest = [i for i in file_list if ('-' + versions[-1]) in i]
        return latest[0]
    else:
        return file_list[0]

In [4]:
enzyme_conc = get_most_updated(enzyme_conc)
cofactor_conc = get_most_updated(cofactor_conc)
metab_conc = get_most_updated(metab_conc)

In [5]:
# get list of all experiments run from buffers_mt
experiments = pd.read_excel(cofactor_conc, index_col=0).index # iloc[:,0]).values

In [6]:
enzyme_df = pd.read_excel(enzyme_conc, index_col=0).dropna(axis=1, how='all')
cofactor_df = pd.read_excel(cofactor_conc, index_col=0).dropna(axis=1, how='all')

In [7]:
def unpivot_df(df, type=None):
    
    
    b=[]
    for col in df.columns: 
        a = pd.DataFrame(df[col])
        a.columns=['conc']
        a['component'] = [col] * len(df)
        if not type:
            a['type']=['enzyme'] * len(df)
        else: 
            a['type']=['cofactor'] * len(df)
        b.append(a)
    return pd.concat(b).dropna().reset_index()

In [21]:
def unpivot_df(df, type=None):
    
    df = df.stack().reset_index()
    df.columns=['experiment', 'component', 'concentration']
    if not type:
        df['type']=['enzyme'] * len(df)
    else: 
        df['type']=['cofactor'] * len(df)
        
    return df

In [22]:
input_data = pd.concat([unpivot_df(cofactor_df, type=1),unpivot_df(enzyme_df)])

pd.pivot_table(input_data, values='concentration', index=['experiment', 'type','component'])

concentration
experiment type     component                 
fdh        cofactor NADP                   1.0
                    fdhAT_QH              10.0
                    formate-koh           25.0
fdh-neg    cofactor NADP                   1.0
                    formate-koh           25.0
...                                        ...
sds-neg    cofactor AMP                    1.0
                    Kan                    1.0
                    PLP                    0.5
                    Serine                 1.0
                    neg                   10.0

[86 rows x 1 columns]

In [8]:
input_data = pd.concat([unpivot_df(cofactor_df, type=1),unpivot_df(enzyme_df)])

pd.pivot_table(input_data, values='conc', index=['index', 'type','component'])

conc
index   type     component        
fdh     cofactor NADP          1.0
                 fdhAT_QH     10.0
                 formate-koh  25.0
fdh-neg cofactor NADP          1.0
                 formate-koh  25.0
...                            ...
sds-neg cofactor AMP           1.0
                 Kan           1.0
                 PLP           0.5
                 Serine        1.0
                 neg          10.0

[86 rows x 1 columns]

In [9]:
compiled_dfs = []
for sheet_name, df in pd.read_excel(metab_conc, index_col=0, sheet_name=None).items():
    
    df = df.stack().reset_index()
    df.drop(['level_1'], axis=1, inplace=True)
    df.columns=['experiment','peak area']
    df['metabolite']=sheet_name
    compiled_dfs.append(df)


In [10]:
all_data = pd.concat(compiled_dfs)
all_data['count']=all_data.groupby(['metabolite', 'experiment']).transform('count')
all_data['median'] = all_data.groupby(['experiment', 'metabolite'])['peak area'].transform('median')
all_data['diff'] = abs(all_data['peak area']-all_data['median'])
all_data['MAD'] = (all_data.groupby(['experiment', 'metabolite']).transform('sum')/all_data.groupby(['experiment', 'metabolite']).transform('count'))['diff']
all_data['cutoff'] = (2*all_data['MAD']) + all_data['median']

cleaned_df = all_data[(all_data['count']>2) & (all_data['peak area'] < all_data['cutoff']) | (all_data['count']<=2)]
cleaned_df

,experiment,peak area,metabolite,count,median,diff,MAD,cutoff
0,fdh,2392.55,malate,2,2651.230,258.680,258.680000,3168.590000
1,fdh,2909.91,malate,2,2651.230,258.680,258.680000,3168.590000
3,fdh-liq,44870.64,malate,3,44870.640,0.000,24011.740000,92894.120000
4,fdh-liq,39225.60,malate,3,44870.640,5645.040,24011.740000,92894.120000
5,fdh-liq-neg,1258.23,malate,3,2656.840,1398.610,1359.316667,5375.473333
...,...,...,...,...,...,...,...,...
29,sds,302.98,nadph,2,294.125,8.855,8.855000,311.835000
30,sds-full,334537.04,nadph,2,266904.360,67632.680,67632.680000,402169.720000
31,sds-full,199271.68,nadph,2,266904.360,67632.680,67632.680000,402169.720000
32,sds-neg,285.08,nadph,2,248.100,36.980,36.980000,322.060000


In [11]:
metabolite_data = cleaned_df[['experiment', 'metabolite', 'peak area']].groupby(['experiment', 'metabolite']).mean()
metabolite_data

peak area
experiment metabolite            
fdh        glycine       6798.090
           malate        2651.230
           nadh           589.155
           nadp        847329.725
           nadph       235525.495
...                           ...
sds-neg    nadph          248.100
           pyr_peak1    24289.945
           pyr_peak2    24338.670
           serine      539524.135
           thf            478.330

[121 rows x 1 columns]

In [1]:
import clean 

In [2]:
EXP_DATE='230821'
clean.import_init_conc_data(EXP_DATE)

concentration
experiment type     component                 
fdh        cofactor NADP                   1.0
                    fdhAT_QH              10.0
                    formate-koh           25.0
fdh-neg    cofactor NADP                   1.0
                    formate-koh           25.0
...                                        ...
sds-neg    cofactor AMP                    1.0
                    Kan                    1.0
                    PLP                    0.5
                    Serine                 1.0
                    neg                   10.0

[86 rows x 1 columns]

In [3]:
clean.import_final_conc_data(EXP_DATE)

peak area
experiment metabolite            
fdh        glycine       6798.090
           malate        2651.230
           nadh           589.155
           nadp        847329.725
           nadph       235525.495
...                           ...
sds-neg    nadph          248.100
           pyr_peak1    24289.945
           pyr_peak2    24338.670
           serine      539524.135
           thf            478.330

[121 rows x 1 columns]